# Vital Parks Script Part 3

## This vital parks script that combines the network from script 1 and the locations from script 2 and then, for each street segment, computes the travel time to the nearest location for each category

# Set Parameters

In [1]:
# This is the assigned walk speed in miles per hr
walk_speed_miles_per_hr = 3.0
walk_speed_feet_per_second = walk_speed_miles_per_hr*1.46667

# Import Packages

In [2]:
import networkx as nx
import pandas as pd 
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from shapely.geometry import Polygon, Point
import requests
import zipfile
import io

In [3]:
# import networkx as nx
# import random
# import matplotlib.pyplot as plt
# from matplotlib.cm import get_cmap
# import pandas as pd 
# import time
# import datetime
# import geopandas as gpd
# import seaborn as sns
# import calendar
# import matplotlib
# from matplotlib.dates import WeekdayLocator, DayLocator, DateFormatter,HourLocator, MO, date2num, num2date
# from matplotlib.ticker import FormatStrFormatter
# from dateutil.relativedelta import relativedelta
# from matplotlib.patches import Rectangle
# from matplotlib.gridspec import GridSpec
# import shapely
# from shapely.geometry import Point, LineString, Polygon, MultiPolygon
# import glob
# import os
# from shapely.geometry import Polygon, Point
# from shapely.ops import unary_union
# import itertools
# import requests
# import json
# import matplotlib.colors as colors
# import numpy as np
# from dotenv import load_dotenv
# import urllib
# import sqlalchemy
# import pyodbc
# import sys
# sys.path.insert(0, 'F:/Projects/IPM_Shared_Code/Python/')
# import geo_functions
# import fiona
# import osmnx as ox
# import momepy

# Import Street Nodes

In [4]:
vital_parks_street_nodes = gpd.read_file('Network_nodes.geojson')

# Import Resources and match to nearest street nodes

In [5]:
category_locations = gpd.read_file('Category_locations.geojson')
category_locations['geometry'] = category_locations.buffer(1)
category_locations = category_locations.explode().reset_index(drop=True)
category_locations = category_locations.reset_index().rename(columns={'index':'amenity_id'})
category_locations['amenity_id'] = category_locations['amenity_id'].astype(str)
category_locations['amenity_id'] = 'amenity_'+category_locations['amenity_id']

amenity_nodes = category_locations[['amenity_id','Category','SubCategoryName','Feature','Future']]

C:\Users\benjamin.mirabelli\AppData\Local\Temp\ipykernel_3892\850212672.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  category_locations = category_locations.explode().reset_index(drop=True)


In [6]:
segmented_locations = category_locations.exterior.segmentize(25)
segmented_locations = segmented_locations.get_coordinates()
segmented_locations['geometry'] = gpd.points_from_xy(segmented_locations.x, segmented_locations.y, crs=2263)
segmented_locations = segmented_locations.reset_index().rename(columns={'index':'amenity_id'})
segmented_locations['amenity_id'] = 'amenity_'+segmented_locations['amenity_id'].astype(str)
segmented_locations = gpd.GeoDataFrame(segmented_locations, geometry='geometry', crs=2263)
segmented_locations = gpd.sjoin_nearest(segmented_locations,vital_parks_street_nodes, how='left', distance_col='dist')
segmented_locations = segmented_locations[['amenity_id','street_node_id','dist']]
segmented_locations = segmented_locations.groupby(['amenity_id','street_node_id']).agg({'dist':'min'}).reset_index()

In [7]:
category_connections = category_locations.merge(segmented_locations, on='amenity_id',how='left')
category_connections['mode'] = 'amenity_walk'
category_connections = category_connections[['amenity_id','street_node_id','mode','dist']].rename(columns={'amenity_id':'source','street_node_id':'target'})
category_connections.loc[:,'dist'] = category_connections.loc[:,'dist']/walk_speed_feet_per_second
category_connections = category_connections.rename(columns={'dist':'weight'})

In [8]:
Network_edges_all = gpd.read_file('Network_edges.geojson')

In [9]:
Network_edges_walk = Network_edges_all[~Network_edges_all['mode'].str.contains('mta')]

In [10]:
Network_edges_all_with_amenities = pd.concat([Network_edges_all,category_connections])

all_attributes = Network_edges_all_with_amenities.columns[~Network_edges_all_with_amenities.columns.isin(['source','target'])].to_list()

DG_all = nx.from_pandas_edgelist(Network_edges_all_with_amenities, create_using=nx.DiGraph(), edge_attr=all_attributes)

In [11]:
Network_edges_walk_with_amenities = pd.concat([Network_edges_all,category_connections])

walk_attributes = Network_edges_walk_with_amenities.columns[~Network_edges_walk_with_amenities.columns.isin(['source','target'])].to_list()

DG_walk = nx.from_pandas_edgelist(Network_edges_walk_with_amenities, create_using=nx.DiGraph(), edge_attr=walk_attributes)

In [12]:
travel_time_nodes = pd.DataFrame(set(Network_edges_walk['source'].unique()).union(Network_edges_walk['target'].unique())).rename(columns={0:'index'})

for subcat in amenity_nodes['SubCategoryName'].unique():
    shortest_path_raw_temp = nx.multi_source_dijkstra(DG_all, amenity_nodes.loc[amenity_nodes['SubCategoryName']==subcat,'amenity_id'].to_list(), cutoff=2*60*60, weight='weight')
    shortest_path_temp = pd.DataFrame([shortest_path_raw_temp[0]]).transpose().rename(columns={0:subcat+'_mta_tt_'})
    shortest_path_temp[subcat+'_mta_amenity_'] = pd.DataFrame([shortest_path_raw_temp[1]]).transpose()[0].apply(lambda x: x[0])
    shortest_path_temp = shortest_path_temp.reset_index()
    travel_time_nodes = travel_time_nodes.merge(shortest_path_temp, how='left', on='index')
    
for subcat in amenity_nodes['SubCategoryName'].unique():
    shortest_path_raw_temp = nx.multi_source_dijkstra(DG_walk, amenity_nodes.loc[amenity_nodes['SubCategoryName']==subcat,'amenity_id'].to_list(), cutoff=2*60*60, weight='weight')
    shortest_path_temp = pd.DataFrame([shortest_path_raw_temp[0]]).transpose().rename(columns={0:subcat+'_walk_tt_'})
    shortest_path_temp[subcat+'_walk_amenity_'] = pd.DataFrame([shortest_path_raw_temp[1]]).transpose()[0].apply(lambda x: x[0])
    shortest_path_temp = shortest_path_temp.reset_index()
    travel_time_nodes = travel_time_nodes.merge(shortest_path_temp, how='left', on='index')
    

travel_source = travel_time_nodes.rename(columns={'index':''}).add_suffix('source')
travel_target = travel_time_nodes.rename(columns={'index':''}).add_suffix('target')

In [13]:
edges_access_table = Network_edges_walk[['uniqueID','source','target','weight']].merge(travel_source, how='left', on='source').merge(travel_target, how='left', on='target')
edges_access_table = edges_access_table.fillna(2*60*60)

for subcat in amenity_nodes['SubCategoryName'].unique():
    edges_access_table.loc[:,'travel_time$'+subcat+'$travel_mode:walk'] = edges_access_table.loc[:,[subcat+'_walk_tt_source',subcat+'_walk_tt_target','weight']].sum(axis=1)/2
    edges_access_table.loc[:,'amenity$'+subcat+'$travel_mode:walk'] = edges_access_table.loc[:,subcat+'_walk_amenity_source']
    edges_access_table.loc[~(edges_access_table[subcat+'_walk_tt_source']<=edges_access_table[subcat+'_walk_tt_target']),'amenity$'+subcat+'$travel_mode:walk'] = edges_access_table.loc[~(edges_access_table[subcat+'_walk_tt_source']<=edges_access_table[subcat+'_walk_tt_target']),subcat+'_walk_amenity_target']
    edges_access_table.loc[edges_access_table['travel_time$'+subcat+'$travel_mode:walk']>=2*60*60,'amenity$'+subcat+'$travel_mode:walk'] = 'not_in_range'


    edges_access_table.loc[:,'travel_time$'+subcat+'$travel_mode:MTA'] = edges_access_table.loc[:,[subcat+'_mta_tt_source',subcat+'_mta_tt_target','weight']].sum(axis=1)/2
    edges_access_table.loc[:,'amenity$'+subcat+'$travel_mode:MTA'] = edges_access_table.loc[:,subcat+'_mta_amenity_source']
    edges_access_table.loc[~(edges_access_table[subcat+'_mta_tt_source']<=edges_access_table[subcat+'_mta_tt_target']),'amenity$'+subcat+'$travel_mode:MTA'] = edges_access_table.loc[~(edges_access_table[subcat+'_mta_tt_source']<=edges_access_table[subcat+'_mta_tt_target']),subcat+'_mta_amenity_target']
    edges_access_table.loc[edges_access_table['travel_time$'+subcat+'$travel_mode:MTA']>=2*60*60,'amenity$'+subcat+'$travel_mode:MTA'] = 'not_in_range'

C:\Users\benjamin.mirabelli\AppData\Local\Temp\ipykernel_3892\3411824708.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  edges_access_table.loc[:,'travel_time$'+subcat+'$travel_mode:walk'] = edges_access_table.loc[:,[subcat+'_walk_tt_source',subcat+'_walk_tt_target','weight']].sum(axis=1)/2
C:\Users\benjamin.mirabelli\AppData\Local\Temp\ipykernel_3892\3411824708.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  edges_access_table.loc[:,'amenity$'+subcat+'$travel_mode:walk'] = edges_access_table.loc[:,subcat+'_walk_amenity_

In [14]:
Segment_travel_time_table = Network_edges_walk[['uniqueID','Street','From','To','CommunityBoard','DAC_ID','DAC_Designation','Precinct', 'GVP','Trie_label','Trie_name','Council_District','Boro','NYPDID','CT2020','CT2020Suf','CB2020','CB2020Suf','length','Geo_ID','population', 'geometry']]

In [15]:
edges_travel_time_table = edges_access_table.loc[:,(['uniqueID'] + edges_access_table.columns[edges_access_table.columns.str.contains('travel_time')].to_list())]

cat_cols = edges_travel_time_table.columns[edges_travel_time_table.columns.str.contains('travel_time')].to_list()
edges_travel_time_table.loc[:,cat_cols] = (edges_travel_time_table.loc[:,cat_cols]/60).astype(int)+1

In [16]:
Segment_travel_time_table = Segment_travel_time_table.merge(edges_travel_time_table, on='uniqueID', how='left')

## Export Travel Time Table

In [ ]:
# ###################################################
# # Upload/save this segment travel time table to SQL server, as a GeoJSON, or however you want to store it so that they can be accessed by the third vital parks script
# ###################################################

# Segment_travel_time_table.to_file('Segment_Travel_Time.geojson', driver='GeoJSON')

# END Vital Parks Script Part 3